# Instruction-tuning/Fine-tuning Llama 2

### Installing Required Libraries

First, we will install some required libraries.

`transformers`: for loading a large language model and fine-tuning it.

`bitsandbytes`: for loading the model in 4-bit precision.

`accelerate`: for training models and performing inference at scale.

`peft`: for fine-tuning a small number of parameters.

`trl`: for training transformer language models using Reinforcement Learning.


In [593]:
#!g1.1
# %pip install -q accelerate==0.21.0 --progress-bar off
# %pip install -q peft==0.4.0 --progress-bar off
# %pip install -q bitsandbytes==0.40.2 --progress-bar off
# %pip install -q transformers==4.31.0 --progress-bar off
# %pip install -q trl==0.4.7 --progress-bar off

%pip install -q accelerate --progress-bar off
%pip install -q peft --progress-bar off
%pip install -q bitsandbytes --progress-bar off
%pip install -q transformers --progress-bar off
%pip install -q trl --progress-bar off

### Loading Required Libraries

Next, we will load the required libraries for fine-tuning a Large Language Model (LLM) like Llama 2. We will look at each imported class in greater detail in subsequent sections.

In [594]:
#!g1.1
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
# from google.colab import drive
# drive.mount('/content/drive')

/home/jupyter/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jupyter/.local/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


### Hugging Face Hub Login

In [592]:
#!huggingface-cli login

Unknown instance spec: huggingface-cli

### Creating Bitsandbytes Configuration


In [595]:
#!g1.1
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

### Loading Hugging Face Model and Tokenizer

We will now define a function `load_model` that accepts the model name (`model_name`) from Hugging Face Hub and the `bitsandbytes` configuration for model quantization.

In this function, we will perform the following steps:
 1. Get the number of GPUs available.
 2. Set the maximum GPU memory.
 3. Use the from_pretrained` method from the `AutoModelForCausalLM` class to load a pre-trained Hugging Face model in 4-bit precision using the model name and the quantization configuration.
 4. Set which device to send the model to using `device_map`. Passing `device_map = 0` means putting the whole model on GPU 0. Other inputs could be `cpu`, `cuda:1`, etc. Setting `device_map = auto` will let `accelerate` compute the most optimized `device_map` automatically.
 5. Set `max_memory`, a dictionary device identifier, to maximum memory, which will default to the maximum memory available for each GPU and the available CPU RAM if unset.
 6. Load the model tokenizer from the model name on Hugging Face.
 7. Set a padding token to ensure shorter sequences will have the same length as the longest sequence in a batch. In this case, we will set the EOS (End of Sentence) token as the padding token.

 **Important Note:  A tokenizer for a model will preprocess and tokenize (convert letters/words/sub-words to tokens or numbers) the input in a way that the model expects. Model tokenizers are also responsible for correctly applying special tokens and certain special embeddings or positional encoders specific to a model in the input.**

In [596]:
#!g1.1
import time
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'
    time.sleep(2)
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    time.sleep(2)
    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = False)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

### Initializing Transformers and Bitsandbytes Parameters

We will now initialize input parameters for the `transformers` and `bitsandbytes` modules.

In [597]:
#!g1.1
################################################################################
# transformers parameters
################################################################################

# The pre-trained model from the Hugging Face Hub to load and fine-tune
model_name = "OpenBuddy/openbuddy-llama2-13b-v8.1-fp16" #"ai-forever/mGPT"


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

Finally, we will call the above functions to get `model` and `tokenizer` objects.

In [598]:
#!g1.1
#!g1.1
# Load model from Hugging Face Hub with model name and bitsandbytes configuration

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

In [599]:
#!g1.1

model, tokenizer = load_model(model_name, bnb_config)

Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.35s/it]
/home/jupyter/.local/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


### Loading Dataset

In [600]:
#!g1.1
# The instruction dataset to use
dataset_name = "classification.csv"

In [601]:
#!g1.1
#del_datasphere_variables('dataset')

In [602]:
#!g1.1
# Load dataset
dataset = load_dataset("csv", data_files = dataset_name, split = "train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 95.96it/s]
Generating train split: 110 examples [00:00, 1366.60 examples/s]
Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1

In [603]:
#!g1.1
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 110
Column names are: ['instruction', 'input', 'output']


Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", li

The `load_dataset` function will convert the CSV file into a dictionary of prompts. We can look at a random prompt in the dataset using a random index.

In [604]:
#!g1.1
dataset[randrange(len(dataset))]

Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", li

{'instruction': 'К какому разделу относится предложение:\\n\\n1\\n2\\n3\\n4\\n5\\n6\\n7\\n8\\n9\\n10\\n11\\n12\\n13\\n14\\n15\\n16\\n17\\n18\\n-1\\n\\n',
 'input': 'VIII. Основные направления развития межрегиональных и внешнеэкономических связей',
 'output': 9}

### Creating Prompt Template

After loading the instruction dataset, we will define the `create_prompt_formats` function to create a prompt template against each prompt in our dataset and save it in a new dictionary key `text` for further data preprocessing and fine-tuning.

In [605]:
#!g1.1
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [606]:
#!g1.1
create_prompt_formats(dataset[randrange(len(dataset))])

Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", li

{'instruction': 'К какому разделу относится предложение:\\n\\n1\\n2\\n3\\n4\\n5\\n6\\n7\\n8\\n9\\n10\\n11\\n12\\n13\\n14\\n15\\n16\\n17\\n18\\n-1\\n\\n',
 'input': 'Пространственное развитие',
 'output': 10,
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nК какому разделу относится предложение:\\n\\n1\\n2\\n3\\n4\\n5\\n6\\n7\\n8\\n9\\n10\\n11\\n12\\n13\\n14\\n15\\n16\\n17\\n18\\n-1\\n\\n\n\nInput:\nПространственное развитие\n\n### Response:\n10\n\n### End'}

### Getting Maximum Sequence Length of the Pre-trained Model

In [607]:
#!g1.1
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

### Tokenizing Dataset Batch

The user-defined `preprocess_batch` function will tokenize a batch of the input dataset (`batch`) using the `tokenizer` object. We will set the maximum sequence length using the `max_length` parameter, which will control the maximum length used by the padding or truncation parameter. `truncation = True` will truncate the input to the maximum length provided by the `max_length` parameter. Similarly, `padding = max_length` will pad the input to the maximum length provided. This function will be called in the `preprocess_dataset` function defined next.

In [608]:
#!g1.1
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

### Preprocessing Dataset

To preprocess the complete dataset for fine-tuning, we will define the `preprocess_dataset` function, which will perform the following operations:

1. Create the formatted prompts against each prompt in the instruction dataset using the `create_prompt_formats` function.
2. Tokenize the dataset in batches using the `preprocess_batch` function and removing the original dictionary keys (instruction, input, output, and text).
3. Filter out prompts with input token sizes exceeding the maximum length.
4. Shuffle the dataset using a random seed.

In [609]:
#!g1.1
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "input", "output", "text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [610]:
#!g1.1
#del_datasphere_variables('preprocessed_dataset')

In [611]:
#!g1.1
# Random seed
seed = 33

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Found max lenth: 2048
Preprocessing dataset...


Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", li

We can now look at the preprocessed dataset, which contains tokens or IDs.

In [612]:
#!g1.1
print(preprocessed_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 110
})


Failed to deserialize variable 'preprocessed_dataset'. Run the following code to delete it:
  del_datasphere_variables('preprocessed_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  Fi

In [613]:
#!g1.1
print(preprocessed_dataset[0])

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 30014, 1787, 21177, 3212, 1216, 1844, 25754, 7489, 4570, 843, 11567, 3583, 29876, 29905, 29876, 29896, 29905, 29876, 29906, 29905, 29876, 29941, 29905, 29876, 29946, 29905, 29876, 29945, 29905, 29876, 29953, 29905, 29876, 29955, 29905, 29876, 29947, 29905, 29876, 29929, 29905, 29876, 29896, 29900, 29905, 29876, 29896, 29896, 29905, 29876, 29896, 29906, 29905, 29876, 29896, 29941, 29905, 29876, 29896, 29946, 29905, 29876, 29896, 29945, 29905, 29876, 29896, 29953, 29905, 29876, 29896, 29955, 29905, 29876, 29896, 29947, 29905, 29876, 29899, 29896, 29905, 29876, 29905, 29876, 13, 13, 4290, 29901, 13, 29896, 29896, 29889, 857, 6711, 730, 1892, 29970, 21944, 30002, 821, 30051, 13335, 13584, 2641, 16551, 5395, 13, 13, 2277, 29937, 13291, 29901, 13, 29929, 13, 13, 2277, 29937, 2796], 'attention_mask': [1, 1, 1, 1, 1

Failed to deserialize variable 'preprocessed_dataset'. Run the following code to delete it:
  del_datasphere_variables('preprocessed_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  Fi

### Creating PEFT Configuration


Fine-tuning pretrained LLMs on downstream datasets results in huge performance gains when compared to using the pretrained LLMs out-of-the-box. However, as models get larger and larger, full fine-tuning becomes infeasible to train on consumer hardware. In addition, storing and deploying fine-tuned models independently for each downstream task becomes very expensive, because fine-tuned models are the same size as the original pretrained model. Parameter-Efficient Fine-tuning (PEFT) approaches are meant to address both problems!


PEFT approaches only fine-tune a small number of (extra) model parameters while freezing most parameters of the pretrained LLMs, thereby greatly decreasing the computational and storage costs. It also helps in portability, wherein users can tune models using PEFT methods to get tiny checkpoints worth a few MB compared to the large checkpoints of full fine-tuning.


**In short, PEFT approaches enable you to get performance comparable to full fine-tuning while only having a small number of trainable parameters.**


Hugging Face provides the PEFT library, which provides the latest Parameter-Efficient Fine-tuning techniques seamlessly integrated with Hugging Face Transformers and Hugging Face Accelerate.


There are several PEFT methods. In the next cell, we will use QLoRA, one of the latest methods that reduces the memory usage of LLM finetuning without performance tradeoffs, using the `LoraConfig` class from the `peft` library.


QLoRA uses 4-bit quantization to compress a pretrained language model. The LM parameters are then frozen, and a relatively small number of trainable parameters are added to the model in the form of Low-Rank Adapters. During finetuning, QLoRA backpropagates gradients through the frozen 4-bit quantized pretrained language model into the Low-Rank Adapters. The LoRA layers are the only parameters being updated during training.

In [614]:
#!g1.1
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

### Finding Modules for LoRA Application

In the next cell, we will define the `find_all_linear_names` function to find the module to apply LoRA to. This function will get the module names from `model.named_modules()` and store it in a set to keep distinct module names.

In [615]:
#!g1.1
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

### Calculating Trainable Parameters

We can use the `print_trainable_parameters` function to find out the number and percentage of trainable model parameters. This function will calculate the number of total parameters in `model.named_parameters()` and then those that would get updated.

In [616]:
#!g1.1
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

### Fine-tuning the Pre-trained Model

We will create `fine_tune`, our final function, to wrap everything we have done so far and initiate the fine-tuning process. This function will perform the following model preprocessing operations to prepare it for training:


1. Enable gradient checkpointing to reduce memory usage during fine-tuning.
2. Use the `prepare_model_for_kbit_training` function from PEFT to prepare the model for fine-tuning.
3. Call find_all_linear_names` to get the module names to apply LoRA to.
4. Create LoRA configuration by calling the `create_peft_config` function.
5. Wrap the base Hugging Face model for fine-tuning to PEFT using the `get_peft_model` function.
6. Print the trainable parameters.


For training, we will instantiate a `Trainer()` object within the `fine_tune` function. This class requires the model, preprocessed dataset, and training arguments, listed below.


`per_device_train_batch_size`: The batch size per GPU/TPU/CPU for training.


`gradient_accumulation_steps`: Number of update steps to accumulate the gradients for, before performing a backward/update pass.


`warmup_steps`: Number of steps used for a linear warmup from 0 to `learning_rate`.


`max_steps`: If set to a positive number, the total number of training steps to perform.


`learning_rate`: The initial learning rate for Adam.


`fp16`: Whether to use 16-bit (mixed) precision training (through NVIDIA apex) instead of 32-bit training.


`logging_steps`: Number of update steps between two logs.


`output_dir`: The output directory where the model predictions and checkpoints will be written.


`optim`: The optimizer to use for training.


Next, we will use the `train` method on the trainer` object to start the training and log and save the model metrics on the training dataset. Finally, we will save the model checkpoint (model weights, configuration file, and tokenizer) in the output directory and delete the model to free up memory. You can load the model for inference later using its saved checkpoint.

In [617]:
#!g1.1
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
            report_to="tensorboard",####???/
            
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

Initializing QLoRA and TrainingArguments parameters below for training.

In [618]:
#!g1.1
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

In [619]:
#!g1.1
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 20 #0

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [620]:
#!g1.1
%pip install tensorboardX

Failed to deserialize variable 'dataset'. Run the following code to delete it:
  del_datasphere_variables('dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", li

Defaulting to user installation because normal site-packages is not writeable


Calling the `fine_tune` function below to fine-tune or instruction-tune the pre-trained model on our preprocessed news classification instruction dataset.

In [621]:
#!g1.1
fine_tune(model,
      tokenizer,
      preprocessed_dataset,
      lora_r,
      lora_alpha,
      lora_dropout,
      bias,
      task_type,
      per_device_train_batch_size,
      gradient_accumulation_steps,
      warmup_steps,
      max_steps,
      learning_rate,
      fp16,
      logging_steps,
      output_dir,
      optim)

Failed to deserialize variable 'preprocessed_dataset'. Run the following code to delete it:
  del_datasphere_variables('preprocessed_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 1049, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 996, in pyarrow.lib.MemoryMappedFile._open
  Fi

Step,Training Loss


Step,Training Loss
1,1.704500


Step,Training Loss
1,1.704500
2,1.842800


Step,Training Loss
1,1.704500
2,1.842800
3,1.785000


...

Step,Training Loss
1,1.704500
2,1.842800
3,1.785000
4,1.722400
5,1.582900
6,1.585500
7,1.573100
8,1.423800
9,1.399700
10,1.473300


Step,Training Loss
1,1.704500
2,1.842800
3,1.785000
4,1.722400
5,1.582900
6,1.585500
7,1.573100
8,1.423800
9,1.399700
10,1.473300


Step,Training Loss
1,1.704500
2,1.842800
3,1.785000
4,1.722400
5,1.582900
6,1.585500
7,1.573100
8,1.423800
9,1.399700
10,1.473300


Step,Training Loss
1,1.704500
2,1.842800
3,1.785000
4,1.722400
5,1.582900
6,1.585500
7,1.573100
8,1.423800
9,1.399700
10,1.473300


LoRA module names: ['k_proj', 'q_proj', 'o_proj', 'gate_proj', 'down_proj', 'v_proj', 'up_proj']
All Parameters: 6,792,238,080 || Trainable Parameters: 62,586,880 || Trainable Parameters %: 0.9214470880266906
Training...
***** train metrics *****
  epoch                    =       0.73
  total_flos               =   913202GF
  train_loss               =     1.4442
  train_runtime            = 0:01:42.79
  train_samples_per_second =      0.778
  train_steps_per_second   =      0.195
{'train_runtime': 102.7905, 'train_samples_per_second': 0.778, 'train_steps_per_second': 0.195, 'total_flos': 980543773962240.0, 'train_loss': 1.4442025542259216, 'epoch': 0.73}
Saving last checkpoint of the model...


With these steps, we have fine-tuned a popular open-source pre-trained model, Llama-2-13B, on an instruction dataset that we created for news classification!

We can see from the log that there are 3,540,389,888 parameters in the model, out of which 39,976,960 are trainable. That's approximately 1% of the total parameters. The model trained for 20 steps and converged at a loss value of 1.4. It is possible that the converged weights are not the best weights. We can fix this by adding `EarlyStoppingCallback` to the `trainer`, which would regularly evaluate the model on a validation dataset and keep only the best weights.

### Merging Weights & Pushing to Hugging Face

After saving the fine-tuned weights, we can create our fine-tuned model by merging the fine-tuned weights and saving it to a new directory with its tokenizer. By performing this step, we can have a memory-efficient, fine-tuned model and tokenizer for inference. We will also push the fine-tuned model and its associated tokenizer to Hugging Face Hub for public usage.


In [622]:
#!g1.1
# Load fine-tuned weights
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = "auto", torch_dtype = torch.bfloat16)
# Merge the LoRA layers with the base model
model = model.merge_and_unload()

# Save fine-tuned model at a new location
output_merged_dir = "results/rucola_llama2_13b/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

# Save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

Loading checkpoint shards: 100%|██████████| 3/3 [00:29<00:00,  9.88s/it]
Removed shared tensor {'model.layers.34.self_attn.q_proj.weight', 'model.layers.34.self_attn.k_proj.weight', 'model.layers.21.self_attn.o_proj.weight', 'model.layers.5.mlp.gate_proj.weight', 'model.layers.22.mlp.down_proj.weight', 'model.layers.39.post_attention_layernorm.weight', 'model.layers.21.self_attn.v_proj.weight', 'model.layers.3.self_attn.v_proj.weight', 'model.layers.11.self_attn.o_proj.weight', 'model.layers.16.mlp.gate_proj.weight', 'model.layers.22.post_attention_layernorm.weight', 'model.layers.21.mlp.down_proj.weight', 'model.layers.20.self_attn.q_proj.weight', 'model.layers.15.mlp.up_proj.weight', 'model.layers.5.post_attention_layernorm.weight', 'model.layers.15.self_attn.v_proj.weight', 'model.layers.26.input_layernorm.weight', 'model.layers.26.self_attn.o_proj.weight', 'model.layers.4.self_attn.v_proj.weight', 'model.layers.31.input_layernorm.weight', 'model.layers.24.post_attention_layernorm.w

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
#!g1.1
model

In [ ]:
#!g1.1
tokenizer

In [ ]:
#!g1.1
# Fine-tuned model name on Hugging Face Hub
new_model = "sahayk/news-classification-18-llama-2-13b"

In [ ]:
#!g1.1
# Push fine-tuned model and tokenizer to Hugging Face Hub
model.push_to_hub(new_model, use_auth_token = True)
tokenizer.push_to_hub(new_model, use_auth_token = True)